In [ ]:
# [Cell 1] - Import standard libraries
import os
import numpy as np
import cv2
import csv
from tqdm import tqdm
import torch

In [ ]:
# [Cell 1] - Set up paths
import os

is_colab = True
if is_colab:
  from google.colab import drive
  drive.mount('/content/drive')
  BASE_PATH = "/content/drive/MyDrive"  # This will use the current directory
  submission_path = os.path.join(BASE_PATH,  "submission.csv")
  save_model = os.path.join(BASE_PATH,  "siamese.pth")
else:
  BASE_PATH = "/kaggle/input/cvproject"
  submission_path = "submission.csv"
  save_model = "siamese.pth"


is_tiny_roma = True
# !unzip "/content/drive/MyDrive/cv-22928-2025-a-project.zip" -d "/content/"

project_folder = "cv-22928-2025-a-project/train"
project_folder = os.path.join(BASE_PATH, project_folder)
src = os.path.join(BASE_PATH, "cv-22928-2025-a-project")

test_csv = os.path.join(BASE_PATH, "cv-22928-2025-a-project/test.csv")


if os.path.exists(project_folder):
    print(f"Successfully found project directory at: {project_folder}")
else:
    print(f"ERROR: Could not find project directory at: {project_folder}")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Please check if the path '{project_folder}' is correct")

KeyboardInterrupt: 

In [ ]:
# [Cell 3] - Install required packages
!pip install git+https://github.com/Parskatt/RoMa
!pip install numpy opencv-python matplotlib tqdm torch torchvision timm

In [ ]:
# [Cell 4] - Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

if torch.cuda.is_available():
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')
    print(f'Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

In [ ]:
from romatch import tiny_roma_v1_outdoor, roma_outdoor
if is_tiny_roma:
  roma_model = tiny_roma_v1_outdoor(device=device)
  num_inputs = 15000
else:
  roma_model = roma_outdoor(device=device)
  num_inputs = 30000

In [ ]:
# [Cell 6] - Helper functions
def flatten_matrix(M, num_digits=8):
    """Convert matrix to string format for submission."""
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

def process_image_pair(img1_path, img2_path, model):
    """Process a single image pair using RoMa."""
    try:
        # Get image dimensions
        img1 = cv2.imread(img1_path)
        img2 = cv2.imread(img2_path)
        if img1 is None or img2 is None:
            return None

        H_A, W_A = img1.shape[:2]
        H_B, W_B = img2.shape[:2]

        # Match using RoMa
        if is_tiny_roma:
          warp, certainty = model.match(img1_path, img2_path)
        else:
          warp, certainty = model.match(img1_path, img2_path, device=device)

        # Sample matches
        matches, certainty = model.sample(warp, certainty)

        # Convert to pixel coordinates
        kptsA, kptsB = model.to_pixel_coordinates(matches, H_A, W_A, H_B, W_B)

        # Find fundamental matrix
        F, mask = cv2.findFundamentalMat(
            kptsA.cpu().numpy(),
            kptsB.cpu().numpy(),
            ransacReprojThreshold=0.2,
            method=cv2.USAC_MAGSAC,
            confidence=0.999999,
            maxIters=10000
        )

        if F is None or F.shape != (3, 3):
            return None

        return F

    except Exception as e:
        print(f"Error in process_image_pair: {str(e)}")
        return None

In [ ]:
# [Cell 7] - Main execution function
def main():
    project_dir = src
    test_csv = os.path.join(project_dir, 'test.csv')

    if not os.path.exists(test_csv):
        print(f"Error: test.csv not found at {test_csv}")
        return

    with open(test_csv) as f:
        reader = csv.reader(f, delimiter=',')
        next(reader)
        test_samples = list(reader)

    F_dict = {}

    for sample_id, batch_id, image_1_id, image_2_id in tqdm(test_samples, desc='Processing samples'):

      img1_path = os.path.join(project_dir, 'test_images', batch_id, f'{image_1_id}.jpg')
      img2_path = os.path.join(project_dir, 'test_images', batch_id, f'{image_2_id}.jpg')

      if not os.path.exists(img1_path) or not os.path.exists(img2_path):
         F_dict[sample_id] = np.zeros((3, 3))
         continue

      F = process_image_pair(img1_path, img2_path, roma_model)
      F_dict[sample_id] = F if F is not None else np.zeros((3, 3))


    with open(submission_path, 'w') as f:
        f.write('sample_id,fundamental_matrix\n')
        for sample_id, F in F_dict.items():
            f.write(f'{sample_id},{flatten_matrix(F)}\n')

    print(f"Submission saved to: {submission_path}")

In [ ]:
# [Cell 8] - Run the pipeline
if __name__ == "__main__":
    main()
    print("Processing complete! Submission file has been created.")

Processing samples:  31%|███       | 5843/18840 [29:33<1:18:40,  2.75it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')